In [1]:
import os
import keras

import numpy as np
import tensorflow as tf
from tqdm import tqdm
print(tf.__version__)

2.9.0


In [2]:
#FOR ANIMALS
# TRAIN_DIR = "D:/VA/animals_new_dataset"
# FOR NUMBERS
TRAIN_DIR = "E:/VA/30words/accept/agrima325.jpg"


In [7]:
SAVE_DIR = 'E:/VA/agrima325.jpg'


In [9]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

def landmarks(img,results):
    
    #face
    mp_drawing.draw_landmarks(
        img,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
#     #pose
#     mp_drawing.draw_landmarks(
#         img,
#         results.pose_landmarks,
#         mp_holistic.POSE_CONNECTIONS,
#         landmark_drawing_spec=mp_drawing_styles
#         .get_default_pose_landmarks_style())
    #left hand
    mp_drawing.draw_landmarks(
        img,
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
    #         landmark_drawing_spec=None,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    # right hand
    mp_drawing.draw_landmarks(
        img,
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
    #         landmark_drawing_spec=None,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

In [10]:
img_array = cv2.imread(TRAIN_DIR)
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:            
    image_height, image_width, _ = img_array.shape
    # Convert the BGR image to RGB before processing.
    results = holistic.process(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))

    img = np.zeros([image_height, image_width,3],dtype=np.uint8)
    img.fill(255)
    landmarks(img,results) # from function
    cv2.imwrite(SAVE_DIR,img)


In [10]:
# for getting just hands and face
IMG_SIZE=128
training_data = []
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
    for category in CATEGORIES:
        path=TRAIN_DIR+"/"+category
        path2=load_path+"/"+category
#         print(path)
        class_num = CATEGORIES.index(category)
        for c_img in tqdm(os.listdir(path)):
            img_array = cv2.imread((path+"/"+c_img))
            
            image_height, image_width, _ = img_array.shape
            # Convert the BGR image to RGB before processing.
            results = holistic.process(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
                    
            annotated_image = img_array.copy()
            
            img = np.zeros([image_height, image_width,3],dtype=np.uint8)
            img.fill(255)
            
            if not (results.left_hand_landmarks or results.right_hand_landmarks):  # exclude saving no hands
                continue
                
            # conditions on one_hand
            if category in one_hand:
                if  (results.left_hand_landmarks or results.right_hand_landmarks):   #include saving either hands
                    landmarks(img,results) # from function
                    img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
                    cv2.imwrite((path2+"/"+c_img),img)
                    training_data.append([img, class_num])
                    
            # condition on two_hands        
            if category in two_hand:
                if  (results.left_hand_landmarks and results.right_hand_landmarks):   #include both hand
                    landmarks(img,results) # from function
                    img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
                    cv2.imwrite((path2+"/"+c_img),img)
                    training_data.append([img, class_num])
                
            
import random
random.shuffle(training_data)
# np.save('F:/VA/onehandtwohand/26words_DSLR_H/training1a.npy', training_data)            

 22%|█████████████████▍                                                               | 86/400 [00:04<00:17, 18.04it/s]


KeyboardInterrupt: 

In [ ]:
X =[]
y =[]
for features, label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

# X = X.astype('float32')
# X /= 255

from keras.utils import np_utils
Y = np_utils.to_categorical(y, cat_len)
np.save(load_path+'X.npy', X)
np.save(load_path+'Y.npy', Y)
print("done")

In [ ]:
# for getting hands and face contour

import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic


#For webcam input:

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    #face
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    #pose
#     mp_drawing.draw_landmarks(
#         image,
#         results.pose_landmarks,
#         mp_holistic.POSE_CONNECTIONS,
#         landmark_drawing_spec=mp_drawing_styles
#         .get_default_pose_landmarks_style())
    #left hand
    mp_drawing.draw_landmarks(
        image,
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
#         landmark_drawing_spec=None,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    # right hand
    mp_drawing.draw_landmarks(
        image,
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
#         landmark_drawing_spec=None,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())


    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))
    if cv2.waitKey(10) & 0xFF == ord('q'):
            break
# close the camera
cap.release()

# close all the opened windows
cv2.destroyAllWindows()